In [ ]:
%%html
<style>
.container {
    width: 100% !important;
    max-width: 100% !important;
}
</style>

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from types import SimpleNamespace

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Concatenate, ReLU
from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.optimizers.legacy import Adam  # Use legacy optimizer for M1/M2 Macs
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import Masking
import tensorflow as tf

from keras.saving import register_keras_serializable

from IPython.display import display, HTML

In [ ]:
# local functions for display

def header(text, level=2, color='cornflowerblue'):
    display(HTML(f'<h{level} style="color:{color};">{text}</h{level}>'))

def full_display(data: pd.DataFrame, index: bool = False):
    display(HTML(data.to_html(index=index)))

In [ ]:
# define configuration

MODEL_DIR=os.path.abspath(os.path.expanduser("models"))

cfg = SimpleNamespace(
    NA_FILL=-1,
    AUTO_ENCODER_EPOCHS=100,
    AUTO_ENCODER_BATCH_SIZE=32,
    AUTO_ENCODER_LEARNING_RATE=0.001,
    MODEL_DIR=MODEL_DIR,
    AUTOENCODER_MODEL_PATH=os.path.join(MODEL_DIR, "autoencoder_model.keras"),
    RF_MODEL_PATH=os.path.join(MODEL_DIR, "rf_imputer.joblib"),
    OVERWRITE_AUTOENCODER=True,
    OVERWRITE_RF=False,
    VERBOSE=1,
    RANDOM_STATE=100,
    RANDOM_SAMPLE_FRACTION=0.2,
    TEST_SIZE=0.2,
    HISTORY_FILE_PATH=os.path.join(MODEL_DIR, "autoencoder_training_history.json")
)

os.makedirs(cfg.MODEL_DIR, exist_ok=True)

cfg.__dict__

In [ ]:
@register_keras_serializable()  # Register the custom layer
class NonNegativeOutputLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        fixed_outputs = tf.nn.relu(inputs[:, :num_fixed_features])  # Enforce non-negative values for fixed features
        unrestricted_outputs = inputs[:, num_fixed_features:]  # Allow unrestricted values for lat/lon
        return tf.concat([fixed_outputs, unrestricted_outputs], axis=1)

    def get_config(self):
        config = super().get_config()  # Get base config if any
        return config

In [ ]:
data = fetch_california_housing()
data.data.shape

In [ ]:
# Load dataset and simulate missing values
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)

target = data.target

# Save original values for comparison
original_df = df.copy()

# Introduce NaNs randomly, excluding Longitude and Latitude
np.random.seed(cfg.RANDOM_STATE)
nan_mask = np.random.rand(*df.shape) < cfg.RANDOM_SAMPLE_FRACTION
nan_mask[:, df.columns.get_loc('Longitude')] = False  # Exclude Longitude
nan_mask[:, df.columns.get_loc('Latitude')] = False   # Exclude Latitude
df[nan_mask] = np.nan

# Separate rows with and without missing values
train_df = df.dropna()
test_df = df[df.isna().any(axis=1)]

df_filled = df.fillna(cfg.NA_FILL)

# Normalize and handle missing values
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(df_filled)

# Transform the data
df_scaled = pd.DataFrame(scaler.transform(df_filled), columns=df_filled.columns)

# Scale train and test sets individually for model input
X_train = scaler.transform(train_df.fillna(cfg.NA_FILL))
X_test = scaler.transform(test_df.fillna(cfg.NA_FILL))

# Define fixed and unrestricted features
fixed_features = [col for col in df.columns if col not in ["Longitude", "Latitude"]]
num_fixed_features = len(fixed_features)

# Learning rate warm-up function
initial_lr = 1e-5
def warmup(epoch, lr):
    if epoch < 5:
        return lr + (initial_lr * 0.2)
    return lr

# Define the autoencoder architecture
input_dim = X_train.shape[1]
input_layer = Input(shape=(input_dim,))

# Encoder
encoded = Dense(256, activation="relu")(input_layer)
encoded = LayerNormalization()(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(128, activation="relu")(encoded)
encoded = LayerNormalization()(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(64, activation="relu")(encoded)
encoded = LayerNormalization()(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(32, activation="relu", name="embedding")(encoded)

# Decoder
decoded = Dense(64, activation="relu")(encoded)
decoded = LayerNormalization()(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(128, activation="relu")(decoded)
decoded = LayerNormalization()(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(256, activation="relu")(decoded)
decoded = LayerNormalization()(decoded)
decoded = Dropout(0.2)(decoded)

# Final output with custom non-negative layer
output_layer = Dense(input_dim)(decoded)
output_layer = NonNegativeOutputLayer()(output_layer)  # Apply custom layer to enforce constraints

# Model definition
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=initial_lr), loss="mse")

# Callbacks
history_file_path = cfg.HISTORY_FILE_PATH
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_warmup = LearningRateScheduler(warmup, verbose=1)

# Training with saving and loading history
if cfg.OVERWRITE_AUTOENCODER or not os.path.exists(cfg.AUTOENCODER_MODEL_PATH):
    print("Training autoencoder with enhanced configuration...")
    history = autoencoder.fit(
        X_train, X_train,
        epochs=cfg.AUTO_ENCODER_EPOCHS,
        batch_size=cfg.AUTO_ENCODER_BATCH_SIZE,
        validation_data=(X_test, X_test),
        verbose=cfg.VERBOSE,
        callbacks=[lr_warmup, lr_reduction, early_stopping]
    )
    autoencoder.save(cfg.AUTOENCODER_MODEL_PATH)

    # Save training history to disk
    with open(history_file_path, 'w') as f:
        json.dump({k: [float(v) for v in values] for k, values in history.history.items()}, f)
    history_data = history.history  # Save for plotting
else:
    print("Loading pre-trained autoencoder...")
    # autoencoder = load_model(cfg.AUTOENCODER_MODEL_PATH)
    autoencoder = load_model(
        cfg.AUTOENCODER_MODEL_PATH,
        custom_objects={"NonNegativeOutputLayer": NonNegativeOutputLayer},
        compile=False
    )
    autoencoder.compile(optimizer=Adam(learning_rate=initial_lr), loss="mse")
    with open(history_file_path, 'r') as f:
        history_data = json.load(f)

In [ ]:
history_long = pd.DataFrame({
    'epoch': list(range(len(history_data['loss']))),
    'training loss': history_data['loss'],
    'validation loss': history_data['val_loss']
}).melt(id_vars='epoch', var_name='type', value_name='loss_value')

sns.set_theme(style="darkgrid")
sns.set_context("talk", font_scale=1.1)

plt.figure(figsize=(12, 8))
sns.lineplot(
    data=history_long,
    x='epoch',
    y='loss_value',
    hue='type',
    palette={'training loss': 'cornflowerblue', 'validation loss': 'red'}
)
plt.yscale("log")
plt.xlabel('epoch')
plt.ylabel('loss (MSE)')
# plt.title('Autoencoder Training and Validation Loss Over Epochs', loc='left')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
legend = plt.legend(title='', loc='upper right', frameon=False)

# Save the plot to a JPEG file
plt.savefig("autoencoder_loss_curves.jpg", format="jpg", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Impute missing values using the autoencoder
print("Imputing missing values using the autoencoder...")
X_test_pred = autoencoder.predict(X_test)
pred_df = pd.DataFrame(scaler.inverse_transform(X_test_pred), columns=df.columns, index=test_df.index)
display(pred_df)

In [ ]:
tf.autograph.set_verbosity(0)  # Suppress retracing warnings


def find_embedding_layer_by_name(model, layer_name="embedding"):
    for layer in model.layers:
        if layer.name == layer_name:
            return layer
    return None


def get_embeddings(autoencoder, input_df, scaler):
    """
    Scales the input dataframe using 'scaler', finds the embedding layer in the
    'autoencoder', then creates a sub-model that outputs the embedding vectors.
    Returns a DataFrame containing the embedding for each row of 'input_df'.
    """
    # Normalize the entire input DataFrame
    normalized_input = scaler.transform(input_df)

    # Find the embedding layer
    embedding_layer = find_embedding_layer_by_name(autoencoder, layer_name="embedding")
    if embedding_layer is None:
        raise ValueError("No embedding layer found. Check model architecture or layer definitions.")

    # Create a sub-model from the autoencoder's input to the embedding layer's output
    encoder_model = Model(inputs=autoencoder.input, outputs=embedding_layer.output)

    # Predict the embeddings (suppressing verbose output)
    embeddings = encoder_model.predict(normalized_input, verbose=0)
    return pd.DataFrame(embeddings, index=input_df.index)


# 1. Sample 3 rows of your data
data = original_df.sample(3, random_state=1000)

# 2. Extract embeddings
embedding_vectors_df = get_embeddings(autoencoder, data, scaler)

# 3. Display original data
header('original data sample')
display(HTML(data.to_html(index=False)))

# 4. Split the embedding columns for display
midpoint = len(embedding_vectors_df.columns) // 2
first_half = embedding_vectors_df.iloc[:, :midpoint]
second_half = embedding_vectors_df.iloc[:, midpoint:]

# 5. Display the first half of the embedding columns
header('autoencoder-generated embeddings (first half)')
display(HTML(first_half.to_html(index=False)))

# 6. Display the second half of the embedding columns
header('autoencoder-generated embeddings (second half)')
display(HTML(second_half.to_html(index=False)))

In [ ]:
if cfg.OVERWRITE_RF or not os.path.exists(cfg.RF_MODEL_PATH):

    print("Imputing missing values using IterativeImputer with Random Forest...")
    
    # Fit the imputer on X_train (complete cases only)
    rf_imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=cfg.RANDOM_STATE)
    rf_imputer.fit(X_train)  # Train only on non-missing data

    # Save the fitted IterativeImputer with RandomForest
    joblib.dump(rf_imputer, cfg.RF_MODEL_PATH)
    print("Imputer model saved to disk.")

else:
    print('Loading RF imputation model from disk')
    rf_imputer = joblib.load(cfg.RF_MODEL_PATH)

# Use the trained imputer to transform X_test, applying the learned imputation patterns
rf_imputed_scaled = rf_imputer.transform(X_test)
rf_imputed = pd.DataFrame(scaler.inverse_transform(rf_imputed_scaled), columns=df.columns, index=test_df.index)

In [ ]:
# Calculate absolute differences between imputed values and original values for the test set
abs_diff_records = []

for col in test_df.columns:
    # Get indices in the test set where original data had NaNs
    missing_indices = test_df.loc[test_df[col].isna(), col].index

    # Extract the relevant values from original, autoencoder, and random forest imputed DataFrames
    original_values = original_df.loc[missing_indices, col]
    ae_imputed_values = pred_df.loc[missing_indices, col]
    rf_imputed_values = rf_imputed.loc[missing_indices, col]
    
    # Calculate absolute differences for each model, avoiding potential NaN comparisons
    ae_abs_diff = (ae_imputed_values - original_values).abs()
    rf_abs_diff = (rf_imputed_values - original_values).abs()
    
    # Append results for plotting
    abs_diff_records.extend([
        {'Feature': col, 'Model': 'Autoencoder', 'Abs Difference': diff}
        for diff in ae_abs_diff.dropna()
    ])
    abs_diff_records.extend([
        {'Feature': col, 'Model': 'Random Forest', 'Abs Difference': diff}
        for diff in rf_abs_diff.dropna()
    ])

# Convert to DataFrame for analysis and plotting
abs_diff_df = pd.DataFrame(abs_diff_records)

In [ ]:
# Set a dark theme and define a color palette
sns.set_theme(style="darkgrid")
sns.set_context("talk", font_scale=1.1)
plt.rcParams['font.family'] = 'Avenir'
custom_palette = {"Autoencoder": "cornflowerblue", "Random Forest": "#ff7f0e"}  # Vibrant colors for models

# Create the box plot with enhanced styling
golden_ratio = 1.6180339887
plt.figure(figsize=(14, 14/golden_ratio))
sns.boxplot(
    data=abs_diff_df, 
    x='Feature', 
    y='Abs Difference', 
    hue='Model', 
    palette=custom_palette,
    width=0.5,  # Narrower boxes
    linewidth=1.5,  # Thicker lines for contrast
    fliersize=3  # Smaller outliers
)

# Log scale for the y-axis and adjusted tick parameters
fontsize=18
plt.yscale("log")
plt.xticks(rotation=45, fontsize=fontsize)
plt.yticks(color="black")

# Titles and labels with color adjustments
# plt.title("Absolute Differences Between Imputed and Original Values by Feature and Model", fontsize=fontsize*1.2, fontweight='bold', 
#           loc='left')
plt.xlabel("")
plt.ylabel("absolute difference (log scale)", color="black", fontsize=fontsize)
plt.legend(title="", fontsize=14, title_fontsize=12, facecolor='white', framealpha=0.8)

# Display the plot
plt.tight_layout()

plt.savefig("ae_vs_rf_imputation_accuracy.jpg", format="jpg", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
mean_abs_diff_df = abs_diff_df.groupby(['Feature', 'Model']).median().reset_index()

# Pivot the DataFrame for easier comparison
pivot_df = mean_abs_diff_df.pivot(index='Feature', columns='Model', values='Abs Difference')

# Calculate the ratio of the Abs Difference for Random Forest to Autoencoder
pivot_df['RF/AE'] = pivot_df['Random Forest'] / pivot_df['Autoencoder']

# Display the result with only the 'Ratio' column
ratio_df = pivot_df[['RF/AE']].reset_index()
ratio_df.columns.name = None

ratio_df.sort_values('RF/AE', ascending=False, inplace=True)

full_display(ratio_df.round(1))